In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import glob
import json
import pickle
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import skimage.io as io
from skimage.transform import resize

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_validate
from sklearn.utils import shuffle
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score,
                             auc, roc_curve, roc_auc_score, cohen_kappa_score, plot_confusion_matrix, plot_roc_curve,
                             plot_precision_recall_curve, precision_recall_fscore_support, precision_recall_curve)
from sklearn.utils import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator, save_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model, load_model, model_from_json
from tensorflow.keras.layers import (Input, InputLayer, Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout, Activation, GlobalAveragePooling2D, InputSpec, Flatten, Concatenate)
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.resnet_v2 import ResNet101V2, preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.metrics import Accuracy, AUC, Precision, Recall
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

### Exploratory Data Analysis: 


In [6]:
BASE_DIR = "../input/augmented-alzheimers-disease-dataset/AG_Alzheimer_s Dataset"
TEST_DIR, TRAIN_DIR = [os.path.join(BASE_DIR, sub_dir) for sub_dir in os.listdir(BASE_DIR)]
print(TRAIN_DIR)
print(TEST_DIR)


In [7]:
classes = os.listdir(TRAIN_DIR)
class2label = {}

for i in range(len(classes)):
    class2label[classes[i]] = i
    
label2class = {v:k for k, v in class2label.items()}

print(class2label)
print("<---------------------------------------------------------------------------------->")
print(label2class)

In [8]:
train_distri = {}
test_distri = {}
print("TRAIN-SET DISTRIBUTION\n")
for c in classes:
    path = os.path.join(TRAIN_DIR, c, "*")
    data = glob.glob(path)
    print(f"Number of {c} Images: {len(data)}")
    train_distri[c] = len(data)
    print("--"*20)

print("TEST-SET DISTRIBUTION\n")
for c in classes:
    path = os.path.join(TEST_DIR, c, "*")
    data = glob.glob(path)
    print(f"Number of {c} Images: {len(data)}")
    test_distri[c] = len(data)
    print("--"*20)
    

In [9]:
categories = []
image_name = []
image_id = []
image_format = []
labels = []
img_path = []
img_height = []
img_width = []

for cat in classes:
    path = os.path.join(TRAIN_DIR, cat, "*")
    data = glob.glob(path)
    for i in range(len(data)):
        h, w, c = cv2.imread(data[i]).shape
        cls , img = data[i].split('/')[-2:]
        img_id , img_format = img.split('.')
        img_height.append(h)
        img_width.append(w)
        categories.append(cls)
        image_name.append(img)
        image_id.append(img_id)
        image_format.append(img_format) 
        labels.append(class2label[cls])
        img_path.append(data[i])

In [11]:
assert len(categories) == len(image_name) == len(image_id) == len(image_id) == len(image_format) == len(labels) == len(img_path) == len(img_height) == len(img_width)

In [12]:
SEED = 42
data = pd.DataFrame(list(zip(image_id, image_name, image_format, categories, labels, img_path, img_height, img_width)))
data.columns = ['image_id' , 'image_name' , 'format' , 'Class' , 'label' , 'image_path' , 'height' , 'width']
data.sample(10 , random_state = SEED)

In [13]:
data.shape

In [14]:
data['Class'].value_counts()

In [15]:
# Checking if there is any null values in the TRAIN dataset:
if data.isnull().sum().sum() == 0:
    print('No Null values present in the Train Dataset')
else:
    print(f'{data.isnull().sum().sum()} null values found in the TRAIN dataset')

In [16]:
# Checking if there is no duplicate values in the TRAIN dataset:
if data.duplicated().sum() == 0:
    print("No duplicated values in the Train Dataset.")
else:
    print(f'{data.duplicated().sum()} Duplicated values found in the TRAIN dataset')

In [17]:
plt.figure(figsize=(8, 8))

sns.countplot(data = data , x = 'Class' , order = data['Class'].value_counts().index)
sns.despine(top=True, right=True, left=True, bottom=False)

plt.xticks(ticks= [0,1,2,3], labels=data['Class'].value_counts().index.to_list(), fontsize=12)
plt.yticks([])
plt.title('Number of Samples per Class' , fontsize=15 , pad=20 , fontstyle='italic' , fontweight = 800)
plt.xlabel('Type of Diagnosis' , fontsize=14 , fontweight=400, labelpad=16)
plt.ylabel('')
ax = plt.gca()

for p in ax.patches:
    ax.annotate("%.1f%%" % (100*float(p.get_height()/data.shape[0])), (p.get_x() + p.get_width() / 2., abs(p.get_height())), ha='center', 
                va='bottom', color='black', xytext=(0,2), rotation='horizontal', textcoords='offset points', fontsize = 12, fontstyle = 'italic')

plt.tight_layout()

In [18]:
sample_data = data.sample(16, random_state = SEED)
sample_imgs = sample_data['image_path'].to_list()

plt.figure(figsize = (12,12))
for i in range(16):
    plt.subplot(4,4, i+1)
    img = cv2.imread(sample_imgs[i])
    plt.title(sample_imgs[i].split('/')[-2], fontsize = 12)
    plt.axis('off')
    plt.imshow(img)

In [19]:
print('Shape of the image : {}'.format(img.shape))
print('Image Height: {}'.format(img.shape[0]))
print('Image Width: {}'.format(img.shape[1]))
print('Image Dimensions/Channels: {}'.format(img.ndim))
print("=="*20)
print('Image size: {}'.format(img.size))
print('Image Data Type: {}'.format(img.dtype))
print("=="*20)
print('Maximum RGB value in this image {}'.format(img.max()))
print('Minimum RGB value in this image {}'.format(img.min()))

In [20]:
mean_val = []
std_val = []
max_val = []
min_val = []

for i in range(data.shape[0]):
    img_arr = img_to_array(load_img(data['image_path'][i]))
    mean_val.append(img_arr.mean())
    std_val.append(img_arr.std())
    max_val.append(img_arr.max())
    min_val.append(img_arr.min())

data['mean_pixel'] = mean_val
data['std_dev'] = std_val
data['max_pixel'] = max_val
data['min_pixel'] = min_val

In [21]:
data['mean_variation'] = data['mean_pixel'].mean() - data['mean_pixel']
data.sample(10)

In [22]:
plt.figure(figsize = (8,5))
sns.violinplot(y = 'mean_pixel' , data = data , x = 'Class')
sns.despine()
plt.xticks(rotation=340)
plt.title('Mean value distribution for all Classes')
plt.savefig("meanValue.png")
plt.show()

In [23]:
plt.figure(figsize = (12, 8))
sns.kdeplot(data = data , x = 'mean_pixel' , hue = 'Class' , shade = True , linewidth = 2.5)
plt.title('Image color mean value distribution')
#plt.savefig("colorMean.png")
plt.show()

In [24]:
plt.figure(figsize = (8,5))
sns.kdeplot(data = data , x = 'max_pixel', hue='Class' , linewidth = 2.5)
plt.title('Image color max value distribution by class')
plt.show()

In [25]:
sns.set(rc={'figure.figsize':(16,8)})
fig = sns.relplot(data=data, x="mean_pixel", y=data['std_dev'],alpha=0.8, kind='scatter',col='Class');
sns.despine(top=True, right=True, left=False, bottom=False)

plt.suptitle('Mean and Standard Deviation of Image Samples', fontsize = 16 , fontweight = 550)
fig.tight_layout(pad=3);

In [26]:
plt.figure(figsize=(20,8))
sns.set(style="ticks", font_scale = 1)
ax = sns.scatterplot(data=data, x="mean_pixel", y=data['std_dev'], hue = 'Class',alpha=0.8);
sns.despine(top=True, right=True, left=False, bottom=False)
plt.xticks(rotation=0,fontsize = 12)
ax.set_xlabel('Image Channel Colour Mean',fontsize = 14,weight = 'bold')
ax.set_ylabel('Image Channel Colour Standard Deviation',fontsize = 14,weight = 'bold')
plt.title('Mean and Standard Deviation of Image Samples', fontsize = 16,weight = 'bold');